<a href="https://colab.research.google.com/github/sabdaaf/learning-ADS/blob/main/week1_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
df = pd.read_csv('adult.csv', header=None)
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [11]:
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
 'hours_per_week', 'native_country', 'income']

In [12]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:
#ngecek missing values
df.isnull().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,0
relationship,0
race,0
sex,0


In [18]:
#ngecek data in general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Encoding

In [19]:
#encoding variabel nominal
df_encoded = pd.get_dummies(df, columns=["workclass", "occupation", "relationship", "race", "sex", "native_country"], drop_first=True)

In [21]:
#encoding variabel ordinal
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()

# Encoding untuk 'education'
df_encoded["education"] = label_enc.fit_transform(df["education"])

# Encoding untuk 'marital_status'
df_encoded["marital_status"] = label_enc.fit_transform(df["marital_status"])

In [22]:
#encoding variabel target
df_encoded["income"] = df["income"].map({"<=50K": 0, ">50K": 1})

In [23]:
#cek hasil encoding
print(df_encoded.head())
print(df_encoded.info())  # Pastikan semua kategori sudah diubah menjadi numerik

   age  fnlwgt  education  education_num  marital_status  capital_gain  \
0   39   77516          9             13               4          2174   
1   50   83311          9             13               2             0   
2   38  215646         11              9               0             0   
3   53  234721          1              7               2             0   
4   28  338409          9             13               2             0   

   capital_loss  hours_per_week  income  workclass_ Federal-gov  ...  \
0             0              40     NaN                   False  ...   
1             0              13     NaN                   False  ...   
2             0              40     NaN                   False  ...   
3             0              40     NaN                   False  ...   
4             0              40     NaN                   False  ...   

   native_country_ Portugal  native_country_ Puerto-Rico  \
0                     False                        False   
1 

In [24]:
#encoding
X = df_encoded.drop(columns=["income"])  # Semua fitur kecuali target
y = df_encoded["income"]  # Target label

In [26]:
#split data
from sklearn.model_selection import train_test_split # Import the train_test_split function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
#memastikan encoding income belajar dengan benar
df_encoded["income"] = df["income"].str.strip()  # Hapus spasi di awal/akhir
df_encoded["income"] = df_encoded["income"].map({"<=50K": 0, ">50K": 1})

In [28]:
print(df_encoded.isna().sum())

age                                0
fnlwgt                             0
education                          0
education_num                      0
marital_status                     0
                                  ..
native_country_ Thailand           0
native_country_ Trinadad&Tobago    0
native_country_ United-States      0
native_country_ Vietnam            0
native_country_ Yugoslavia         0
Length: 82, dtype: int64


Modeling

In [30]:
#modeling random forest
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
X = df_encoded.drop(columns=["income"])  # Semua fitur
y = df_encoded["income"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [31]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Prediksi pada data uji
y_pred = model.predict(X_test)

# Evaluasi performa
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Laporan klasifikasi (precision, recall, f1-score)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8620
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4942
           1       0.75      0.64      0.69      1571

    accuracy                           0.86      6513
   macro avg       0.82      0.79      0.80      6513
weighted avg       0.86      0.86      0.86      6513

Confusion Matrix:
[[4614  328]
 [ 571 1000]]


In [32]:
# prompt: make new model with decision tree

from sklearn.tree import DecisionTreeClassifier

# Modeling
# modeling decision tree
X = df_encoded.drop(columns=["income"])  # Semua fitur
y = df_encoded["income"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTreeClassifier(random_state=42)  # Gunakan DecisionTreeClassifier
model.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = model.predict(X_test)

# Evaluasi performa
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Laporan klasifikasi (precision, recall, f1-score)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8228
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      4942
           1       0.63      0.64      0.64      1571

    accuracy                           0.82      6513
   macro avg       0.76      0.76      0.76      6513
weighted avg       0.82      0.82      0.82      6513

Confusion Matrix:
[[4351  591]
 [ 563 1008]]
